# 라이브러리 임포트 및 구글 등

In [1]:



!pip install webdriver_manager

In [156]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [154]:
!sudo apt-get install poppler-utils


Password:
sudo: a password is required


In [2]:
!pip install clipboard

In [3]:
!pip install bs4

In [4]:
!pip install requests

In [5]:
!pip install konlpy

In [6]:
from konlpy.tag import *

In [7]:
!pip install selenium
from selenium import webdriver


In [159]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from wordcloud import WordCloud
import re
from collections import Counter
import pickle
import PyPDF2
from inspect import getfile
import os
from urllib import request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import requests

# 크롤링

In [9]:
mother_path = "https://m.stock.naver.com/research/company/"
#index_number = 0:62015
test_index = 62016

In [ ]:
#content > div.ResearchContent_article__17jRI.fs3 > div.ResearchContent_textArea__3DD_r > div > p:nth-child(2)

In [17]:
full_path = mother_path + str(test_index)
full_path

'https://m.stock.naver.com/research/company/62016'

In [24]:
driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")
driver.implicitly_wait(3)
driver.get(full_path)
#html = driver.page_source
#soup = BeautifulSoup(html, "html.parser")
text_list = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/div[3]/div/p[2]")
test = text_list.text


/var/folders/9t/mmn5mnpn14x8qvd1llmf0s5h0000gn/T/ipykernel_43873/1030809245.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")


KeyboardInterrupt: 

In [18]:
#전체 코드 작성
#2013년 1월 리서치 부터
text_dic = {}
full_list = []
false_index_list = []
driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")
for index in tqdm(range(20000, 62000)):
    path = mother_path+str(index)
    driver.implicitly_wait(2)
    driver.get(path)
    try:
       # company_name = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/div[1]/em/text()")
        text_list = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/div[3]/div")
        text = text_list.text
        full_list.append(text)
       # text_dic[company_name] = text
    except:
        false_index_list.append(index)

/var/folders/9t/mmn5mnpn14x8qvd1llmf0s5h0000gn/T/ipykernel_45685/1530455212.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")
100%|███████████████████████████████████| 42000/42000 [6:16:38<00:00,  1.86it/s]


In [62]:
#full_list[0][7:]
"""
ex = full_list[0][7:]

words = re.compile("[가-힣]+").findall(ex)

sentence = ""
for word in words:
    sentence += " "+ word
sentence
"""

' 비철강부문 계열사에 주목 필요 작년 실적 부진은 어느 정도 예상이 되었으며 제품가격 하락을 주 원인으로 판단한다 판매량은 수요산업이 부진했던 후판 및 일부 제품을 제외하고 대부분 품목에서 증가세를 보여 총 만톤 을 기록했으나 판가하락 영향이 더 컸던 것으로 보인다 단기 시황개선 전망에 따른 수혜 기대'

In [53]:
cnt = Counter()
komoran = Komoran()
for idx in tqdm(range(len(full_list))):
    sentence = ""
    dummy = full_list[idx][7:]
    words = re.compile("[가-힣]+").findall(dummy)
    
    for word in words:
        sentence += " "+word
    noun_list = komoran.nouns(sentence)
    
    for noun in noun_list:
        cnt[noun] += 1
    
    
    

'  비철강부문 계열사에 주목 필요\n  작년 실적 부진은 어느 정도 예상이 되었으며 제품가격 하락을 주 원인으로 판단한다.판매량은 수요산업이 부진했던 후판 및 일부 제품을 제외하고 대부분 품목에서 증가세를 보여 총 3,505만톤(+2%yoy)을 기록했으나 판가하락 영향이 더 컸던 것으로 보인다.\n\n\n단기 시황개선 전망에 따른 수혜 기대.pdf 559KB'

In [13]:
#komoran = Komoran()
#okt.nouns(sentence)
cnt.most_common(10)

In [14]:
komoran.nouns("안녕하세요 저는 사람입니다")

['안녕하세요', '사람']

In [22]:
with open("/Users/infected4098/Desktop/fulllist.pkl", "wb") as f:
    pickle.dump(full_list, f)

In [23]:
with open("/Users/infected4098/Desktop/falselist.pkl","wb") as f_2:
    pickle.dump(false_index_list, f_2)

# 기업 재무정보 추출

In [10]:
with open("/Users/infected4098/Desktop/fulllist.pkl", "rb") as f:
    full_list = pickle.load(f)
full_list[:5]

['[요약]\n    비철강부문 계열사에 주목 필요\n  작년 실적 부진은 어느 정도 예상이 되었으며 제품가격 하락을 주 원인으로 판단한다.판매량은 수요산업이 부진했던 후판 및 일부 제품을 제외하고 대부분 품목에서 증가세를 보여 총 3,505만톤(+2%yoy)을 기록했으나 판가하락 영향이 더 컸던 것으로 보인다.\n\n\n단기 시황개선 전망에 따른 수혜 기대.pdf 559KB',
 '[요약]\n    ▶ 결론: 투자의견 Buy, 목표주가 400,000원 유지\n  - 석유화학 제품의 완만한 스프레드 확대\n- 정보전자 신사업 확대로 성장성 강화\n\n\n4Q12 실적은 선방. 점진적인 회복 단계로!.pdf 441KB',
 '[요약]\n    ▶ 결론: 투자의견 Buy(유지), 목표주가 42만원(유지)\n  - 4Q12 실적은 판매량 감소와 제품가격 하락으로 부진했으나 예상치에 부합하는 수준\n- 상반기 철강업황 반등, 4Q12 바닥으로 2Q13까지 이익모멘텀 감안시 봄성수기 전후까지 trading 관점 매수접근은 여전히 유효\n\n\n분기실적바닥 확인, 연결실적 성장에 주목.pdf 481KB',
 '[요약]\n    ▶ 결론: 투자의견 Buy 및 목표주가 20만원 유지\n  삼성 SDI에 대해 투자의견 Buy 및 목표주가 20만원 유지. 4분기는 노트북 수요 감소로 인한 이차전지 수익성 하락과 자동차용 전지 부문의 적자 반영 확대에 따라 전분기 대비 큰 폭으로 감익. 1분기는 폴리머 출하량 증가에 따른 전지 수익성 회복 전망.\n\n\n폴리머의 외로운 성장.pdf 326KB',
 '[요약]\n    4분기 실적 시장기대치 하회\n  동사의 4분기 실적은 매출액 1조 4,142억원(-5.9%, QoQ), 영업이익 7억원(-99.2%, QoQ)으로 IBK 예상 및 시장 컨센서스를 하회하였다. 4분기 노트북 수요 부진으로 원형전지 출하량이 크게 감소하였고, 폴리머 전지 출하량도 부진하면서 2차 전지 영업이익률 7% 수준을 기록했기 때문이다.\n\n\n4분기 실적 시

In [11]:
len(full_list)

38078

In [16]:
double_list = [re.compile("[가-힣]+").findall(words) for words in full_list]
cnt_count = Counter()
empty_list = []
komoran = Komoran()
for sentence in tqdm(double_list):
    for word in sentence:
        noun_list = komoran.nouns(word)
        for noun in noun_list:
            cnt_count[noun] += 1
    

100%|████████████████████████████████████| 38078/38078 [03:16<00:00, 193.41it/s]


In [18]:
cnt_count.most_common(30)

[('원', 115557),
 ('이익', 73187),
 ('영업', 64247),
 ('분기', 64175),
 ('실적', 56875),
 ('년', 50380),
 ('것', 50246),
 ('증가', 39792),
 ('전망', 36627),
 ('예상', 35590),
 ('매출', 35371),
 ('대비', 33776),
 ('주가', 33124),
 ('매출액', 31034),
 ('성장', 27038),
 ('개선', 25863),
 ('투자', 24460),
 ('시장', 24377),
 ('등', 23552),
 ('목표', 22354),
 ('기록', 21941),
 ('유지', 21454),
 ('감소', 20332),
 ('추정', 19946),
 ('부문', 18382),
 ('상승', 17049),
 ('기대', 16826),
 ('월', 16624),
 ('사업', 16589),
 ('수준', 16017)]

In [14]:
"""
komoran = Komoran()
komoran.nouns("수요산업이")

    path = mother_path+str(index)
    driver.implicitly_wait(2)
    driver.get(path)
    try:
       # company_name = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/div[1]/em/text()")
        text_list = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/div[3]/div")
"""

['수요', '산업']

In [146]:
final_row

['매출액',
 '영업이익',
 '당기순이익',
 '지배주주순이익',
 '비지배주주순이익',
 '영업이익률',
 '순이익률',
 'ROE',
 '부채비율',
 '당좌비율',
 '유보율',
 'EPS',
 'PER',
 'BPS',
 'PBR',
 '주당배당금']

In [129]:
final_row = row.split("\n")

In [134]:
np.array(body).reshape(-1, len(col))

array([[2.304009e+06, 2.368070e+06, 2.796048e+06, 3.054876e+06],
       [2.776850e+05, 3.599390e+05, 5.163390e+05, 4.632790e+05],
       [2.173890e+05, 2.640780e+05, 3.990740e+05, 3.710120e+05],
       [2.150510e+05, 2.609080e+05, 3.924380e+05, 0.000000e+00],
       [2.338000e+03, 3.170000e+03, 6.637000e+03, 0.000000e+00],
       [1.205000e+01, 1.520000e+01, 1.847000e+01, 1.517000e+01],
       [9.440000e+00, 1.115000e+01, 1.427000e+01, 1.214000e+01],
       [8.690000e+00, 9.980000e+00, 1.392000e+01, 1.167000e+01],
       [3.412000e+01, 3.707000e+01, 3.992000e+01, 0.000000e+00],
       [2.335700e+02, 2.148200e+02, 1.967500e+02, 0.000000e+00],
       [2.885602e+04, 3.069279e+04, 3.314362e+04, 0.000000e+00],
       [3.166000e+03, 3.841000e+03, 5.777000e+03, 5.374000e+03],
       [1.763000e+01, 2.109000e+01, 1.355000e+01, 1.029000e+01],
       [3.752800e+04, 3.940600e+04, 4.361100e+04, 4.846200e+04],
       [1.490000e+00, 2.060000e+00, 1.800000e+00, 1.140000e+00],
       [1.416000e+03, 2.9

In [81]:
colnames = re.sub("[가-힣]+", "", col)
colnames_2 =re.sub("\n3", "", colnames)
colnames_2 = re.sub(" ", "", colnames_2)
colnames_2 = colnames_2[1:]
colnames_2.split("\n")
final_col = colnames_2
#rownames_3 = re.sub("\n", "", rownames_2)
#rownames_3 = rownames_3.sep(by = "\t")

['2019.12.', '2020.12.', '2021.12.', '2022.12.']

In [107]:
body_2 = body.split(" ")
num_list = []
for bodies in body_2:
    new_split = bodies.split("\n")
    num_list.append(new_split)
    """
    if len(new_split)> 1:
        for elem in new_split:
            num_list.append(elem)
    else:
        num_list.append(new_split[0])
    num_list.append(new_split)
    """
num_list[:20]  
#body_3 = body_2.split("\n")

[['2,304,009'],
 ['2,368,070'],
 ['2,796,048'],
 ['3,054,876', '277,685'],
 ['359,939'],
 ['516,339'],
 ['463,279', '217,389'],
 ['264,078'],
 ['399,074'],
 ['371,012', '215,051'],
 ['260,908'],
 ['392,438'],
 ['-', '2,338'],
 ['3,170'],
 ['6,637'],
 ['-', '12.05'],
 ['15.20'],
 ['18.47'],
 ['15.17', '9.44'],
 ['11.15']]

In [121]:
"""
raw_numbers = []
for num in num_list:
    if len(num) > 1 :
        for real_num in num:
            try:
                raw_numbers.append(float(real_num.replace(",", "")))
            except:
                raw_numbers.append(0)
    else:
        try:
            
            raw_numbers.append(float(num[0].replace(",", "")))
        except:
            raw_numbers.append(float(num[0].replace("-", "0")))
        
raw_numbers
"""

'\nraw_numbers = []\nfor num in num_list:\n    if len(num) > 1 :\n        for real_num in num:\n            try:\n                raw_numbers.append(float(real_num.replace(",", "")))\n            except:\n                raw_numbers.append(0)\n    else:\n        try:\n            \n            raw_numbers.append(float(num[0].replace(",", "")))\n        except:\n            raw_numbers.append(float(num[0].replace("-", "0")))\n        \nraw_numbers\n'

In [63]:
print(col)

매출액
영업이익
당기순이익
지배주주순이익
비지배주주순이익
영업이익률
순이익률
ROE
부채비율
당좌비율
유보율
EPS
PER
BPS
PBR
주당배당금


In [20]:
raw = requests.get(full_path)
raw_1 = raw.content.decode("utf-8", "replace")
html = BeautifulSoup(raw_1, 'html.parser')
#html.find_all("script")
new_path = "https://www.inflearn.com/questions/535839/dfs-%EC%97%90%EB%9F%AC%EA%B0%80-%EB%82%A9%EB%8B%88%EB%8B%A4"

In [ ]:
java_link = "https://ssl.pstatic.net/tveta/libs/glad/prod/gfp-core.js"
java = requests.get(java_link).text
BeautifulSoup(java, "html")

In [ ]:
#content > div.ResearchContent_article__17jRI.fs3 > div.ResearchContent_textArea__3DD_r > div > p:nth-child(2)

In [ ]:
raw_new = requests.get(new_path).text
html_new = BeautifulSoup(raw_new, "html")
html_new

In [147]:
#네이버 증권 재무정보 크롤링. 사용자가 종목 명 입력했을 때 
def find_corp_by_code(code):
    
    
    driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")
    search_link = "https://m.stock.naver.com/domestic/stock/"+str(code)+"/finance/summary"
    driver.implicitly_wait(3)
    driver.get(search_link)
    #body
    elem = driver.find_element("xpath", "/html/body/div/div[1]/div[4]/div[3]/div/div[1]/ul/li[2]/a")
    elem.click()
    body = driver.find_element("xpath", "/html/body/div/div[1]/div[4]/div[3]/div/div[2]/div[3]/div[1]/table/tbody")
    body = body.text
    driver.implicitly_wait(2)
    body_2 = body.split(" ")
    num_list = []
    for bodies in body_2:
        new_split = bodies.split("\n")
        num_list.append(new_split)
    raw_numbers = []
    for num in num_list:
        if len(num) > 1 :
            for real_num in num:
                try:
                    raw_numbers.append(float(real_num.replace(",", "")))
                except:
                    raw_numbers.append(0)
        else:
            try:
            
                raw_numbers.append(float(num[0].replace(",", "")))
            except:
                raw_numbers.append(float(num[0].replace("-", "0")))
    #0은 그냥 결측값 대체한 것 뿐. 의미없음.
    #columns
    col = driver.find_element("xpath", "/html/body/div/div[1]/div[4]/div[3]/div/div[2]/div[3]/div[1]/table/thead/tr")
    col = col.text
    colnames = re.sub("[가-힣]+", "", col)
    colnames_2 =re.sub("\n3", "", colnames)
    colnames_2 = re.sub(" ", "", colnames_2)
    colnames_2 = colnames_2[1:]
    final_col = colnames_2.split("\n")
    #rows
    
    driver.implicitly_wait(2)
    row = driver.find_element("xpath", "/html/body/div/div[1]/div[4]/div[3]/div/div[2]/div[3]/div[1]/div/table/tbody")
    row = row.text
    final_row = row.split("\n")
    
    
    #df
    num_array = np.array(raw_numbers).reshape(-1, len(final_col))
    num_df = pd.DataFrame(num_array)
    num_df.columns = final_col
    num_df.index = final_row

    return num_df

    """
    code = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/ul/li[1]/div[1]/span/span[1]")
    driver.implicitly_wait(2)
    return code
    """


In [148]:
#삼성전자
find_corp_by_code("005930")

/var/folders/9t/mmn5mnpn14x8qvd1llmf0s5h0000gn/T/ipykernel_60809/628584173.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")


,2019.12.,2020.12.,2021.12.,2022.12.
매출액,2304009.00,2368070.00,2796048.00,3054876.00
영업이익,277685.00,359939.00,516339.00,463279.00
당기순이익,217389.00,264078.00,399074.00,371012.00
지배주주순이익,215051.00,260908.00,392438.00,0.00
비지배주주순이익,2338.00,3170.00,6637.00,0.00
영업이익률,12.05,15.20,18.47,15.17
순이익률,9.44,11.15,14.27,12.14
ROE,8.69,9.98,13.92,11.67
부채비율,34.12,37.07,39.92,0.00
당좌비율,233.57,214.82,196.75,0.00


In [149]:
#메가스터디교육
find_corp_by_code("215200")

/var/folders/9t/mmn5mnpn14x8qvd1llmf0s5h0000gn/T/ipykernel_60809/628584173.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")


,2019.12.,2020.12.,2021.12.,2022.12.
매출액,4373.00,4747.00,7039.00,8500.00
영업이익,596.00,328.00,990.00,1416.00
당기순이익,484.00,210.00,815.00,1042.00
지배주주순이익,470.00,215.00,812.00,0.00
비지배주주순이익,14.00,-5.00,4.00,0.00
영업이익률,13.64,6.91,14.07,16.67
순이익률,11.07,4.42,11.58,12.25
ROE,20.45,8.38,26.83,27.87
부채비율,72.74,100.05,104.53,0.00
당좌비율,33.23,48.85,55.79,0.00


#pdf to json

In [160]:
"""
sample = open("https://ssl.pstatic.net/imgstock/upload/research/company/1673830393946.pdf", "rb")
pdfreader=PyPDF2.PdfFileReader(sample)
x = pdfreader.numPages
"""

'\nsample = open("https://ssl.pstatic.net/imgstock/upload/research/company/1673830393946.pdf", "rb")\npdfreader=PyPDF2.PdfFileReader(sample)\nx = pdfreader.numPages\n'

In [161]:
def get_download(url, fname, directory):
    try:
        os.chdir(directory)
        request.urlretrieve(url, fname)
        print('다운로드 완료\n')
    except HTTPError as e:
        print('error')
        return

In [162]:
#get_download("https://ssl.pstatic.net/imgstock/upload/research/company/1673830393946.pdf", "samsung.pdf", "/Users/infected4098/Desktop/DL/Research files")

다운로드 완료



In [ ]:
#download all pdfs
mother_path = "https://m.stock.naver.com/research/company/"
#index_number = 0:62015
test_index = 62016
driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")
false_download = []
for i in tqdm(range(46000, test_index)):
    full_path = mother_path+str(i)
    try:
        driver.get(full_path)
        driver.implicitly_wait(2)
        elem = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/div[4]/a")
        pdf_link = str(elem.get_attribute("href"))
        corp_info = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/div[1]/em").text
        corp_name = corp_info[:-6]
        date = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/span[2]").text
        filename = str(corp_name)+str(date)+".pdf"
        get_download(pdf_link, filename, "/Users/infected4098/Desktop/DL/Research files" )
    except:
        false_download.append(i)

/var/folders/9t/mmn5mnpn14x8qvd1llmf0s5h0000gn/T/ipykernel_60809/2585688054.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "/Users/infected4098/Desktop/DL/chromedriver")
  0%|                                      | 2/16016 [00:13<32:28:48,  7.30s/it]

다운로드 완료



  0%|                                      | 3/16016 [00:15<20:50:54,  4.69s/it]

다운로드 완료



  0%|                                      | 4/16016 [00:42<60:13:47, 13.54s/it]

...

다운로드 완료



  8%|██▋                              | 1292/16016 [1:32:45<81:23:19, 19.90s/it]

다운로드 완료



  8%|██▋                              | 1293/16016 [1:32:52<65:28:57, 16.01s/it]

다운로드 완료

